<a href="https://colab.research.google.com/github/YunusKaratepe/duygu-tanima/blob/main/application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Must

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# navigate your folder that u wanna save spectrograms.
%cd /content/drive/MyDrive/colab/final_project/app

/content/drive/MyDrive/colab/final_project/app


In [ ]:
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.models import load_model

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC

import cv2
import pickle


In [ ]:
emotions_dict = {
        0: 'Neutral',
        1: 'Calm',
        2: 'Happy',
        3: 'Sad',
        4: 'Angry',
        5: 'Fearful',
        6: 'Disgusted',
        7: 'Surprised'
    }

In [ ]:
def plot_spectrogram(Y, sr, hop_length, y_axis="log"):
    plt.figure(figsize=(25, 10))
    plt.axis('off')
    librosa.display.specshow(Y,
                             sr=sr,
                             hop_length=hop_length,
                             y_axis=y_axis)

    plt.savefig("./spectrogram.png", bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
def create_spectrogram(order: int, spec_type: str):
    voice, sr = librosa.load(audio_file)
    after_stft = librosa.stft(voice, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH)
    abs_stft_result = np.abs(after_stft) ** 2
    log_stft_result = librosa.power_to_db(abs_stft_result)
    if order != 0:
        log_stft_delta = librosa.feature.delta(log_stft_result, order)
        plot_spectrogram(log_stft_delta, sr, HOP_LENGTH, y_axis=spec_type)
        return
    plot_spectrogram(log_stft_result, sr, HOP_LENGTH, y_axis=spec_type)

In [ ]:
def predict_emotion(cnn_model_path, spec_type='log', test_path="none"):

    if not os.path.exists("./spectrogram.png"):
        print("Please first run Create Spectrogram section.")
        return
    if not os.path.exists(cnn_model_path):
        print("Model path does not exists.")
        return


    cnn = load_model(cnn_model_path)

    #spec = cv2.imread("./spectrogram.png")
    # spec = cv2.imread(test_path)

    # spec = cv2.resize(spec, (128, 128))

    if test_path != "none":
        spec = tf.keras.preprocessing.image.load_img(test_path, target_size=(128, 128))

    else:
        spec = tf.keras.preprocessing.image.load_img("./spectrogram.png", target_size=(128, 128))




    spec = np.reshape(spec, [1, 128, 128, 3])


    classes = cnn.predict(spec)
    
    return emotions_dict[list(classes[0]).index(max(list(classes[0])))]   

In [ ]:
def predict_emotion_ml(cnn_model_path, spec_type='log', ml_algorithm='SVC_Polynomial_Kernel', test_path="none"):

    ml_model_path = "/content/drive/MyDrive/colab/final_project/models/ml_models/" + ml_algorithm + ".sav"
    if not os.path.exists(ml_model_path):
        print("ML-Model type is not valid.")
        return
    if not os.path.exists("./spectrogram.png"):
        print("Please first run Create Spectrogram section.")
        return
    if not os.path.exists(cnn_model_path):
        print("Model path does not exists.")
        return

    cnn = load_model(cnn_model_path)
    
    
    if test_path != "none":
        spec = tf.keras.preprocessing.image.load_img(test_path, target_size=(128, 128))

    else:
        spec = tf.keras.preprocessing.image.load_img("./spectrogram.png", target_size=(128, 128))


    spec = np.reshape(spec, [1, 128, 128, 3])
    
    outputLayer = cnn.layers[-5]
    intermediate_layer_model = Model(inputs=cnn.input,
                                    outputs=outputLayer.output)

    intermediate_output = intermediate_layer_model.predict(spec)


    ml_model = pickle.load(open(ml_model_path, 'rb'))

    # ---------------------------------------------- problematik

    
    # ml_output = ml_model.predict_proba(lst)
    # ml_list = list(ml_output)

    for i in range(1,9):
        result = ml_model.score(intermediate_output, [i])
        # print(str(int(result)), end="\t")
        if result > 0.6:
            return emotions_dict[int(i)-1]
    

    # emotions_dict[ml_list.index(max(ml_list))]
    

# Config

In [ ]:
# You can play with these.
spec_type = 'log'
order = 0
cnn_model_path = "/content/drive/MyDrive/colab/final_project/models/seed100/log_specAugment.h5"

# Not recommended to change these.
FRAME_SIZE = 2048
HOP_LENGTH = 512

# Run

In [ ]:
audio_file = './mutlu.mp3'
ipd.Audio(audio_file)

1) Create Spectrogram (do not change parameters from here)

In [ ]:
create_spectrogram(order=order, spec_type=spec_type) # ignore warning

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


2) Predict (do not change parameters from here)

In [ ]:
p = '/content/drive/MyDrive/colab/final_project/spec_data/LogSpectrogram/LogSpectrogramOriginal/07-disgust/Log_05_07_45.png'
predict_emotion(cnn_model_path=cnn_model_path, spec_type=spec_type)

'Sad'

In [ ]:
# p = '/content/drive/MyDrive/colab/final_project/spec_data/LogSpectrogram/LogSpectrogramOriginal/02-calm/Log_01_02_05.png'

# dir = os.listdir("/content/drive/MyDrive/colab/final_project/spec_data/LogSpectrogram/LogSpectrogramOriginal/06-fearful/")
# for filename in dir:
#     p = os.path.join("/content/drive/MyDrive/colab/final_project/spec_data/LogSpectrogram/LogSpectrogramOriginal/06-fearful/", filename)
#     print(predict_emotion_ml(cnn_model_path=cnn_model_path, spec_type=spec_type, ml_algorithm='LinearDiscriminantAnalysis', test_path=p))

predict_emotion_ml(cnn_model_path=cnn_model_path, spec_type=spec_type, ml_algorithm='SVC_Polynomial_Kernel')

'Sad'